# LangChain Expression Language (LCEL)

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
!pip install --upgrade langchain
!pip install pydantic==2.7.4


In [4]:
!pip install langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.5 MB 2.6 MB/s eta 0:00:01
   -------------------- ------------------- 1.3/2.5 MB 2.5 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 2.8 MB/s eta 0:00:00

   ------------- -------------------------- 1/3 [pydantic-settings]
   ------------- -------------------------- 1/3 [pydantic-settings]
   -------------------------- ------------- 2/3 [langchain-community]
   -------------------------- ------------- 2/3 [langchain-community]
   -------------------------- ------------- 2/3 [langchain-community]
   -------------------------- ------------- 2/3 [langchain-community]
   -------------------------- ------------- 2/3 [langchain-community]
   -------------------------- ------------- 2/3 [langchain-community]
 

In [7]:
!pip install -U langchain-openai

In [53]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [57]:
prompt = ChatPromptTemplate.from_template(
    "suggest me an online site on which one can find {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [58]:
chain = prompt | model | output_parser

In [60]:
chain.invoke({"topic": "birds"})

'One online site where you can find information on birds is eBird (www.ebird.org). eBird is a global database of bird sightings, providing information on bird species, distribution, and abundance. You can also use the site to record your own bird sightings and contribute to citizen science research on bird populations.'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [61]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

In [62]:
# !pip install docarray

In [63]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["your mom is a doctor", "bees are important for the environment", "birds can fly"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [64]:
retriever.invoke("what is the occupation of your mom?")

[Document(metadata={}, page_content='your mom is a doctor'),
 Document(metadata={}, page_content='bees are important for the environment'),
 Document(metadata={}, page_content='birds can fly')]

In [65]:
retriever.invoke("birds are capable of?")

[Document(metadata={}, page_content='birds can fly'),
 Document(metadata={}, page_content='bees are important for the environment'),
 Document(metadata={}, page_content='your mom is a doctor')]

In [66]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [67]:
from langchain_core.runnables import RunnableMap

In [70]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [71]:
chain.invoke({"question": "bees?"})

'Bees are important for the environment.'

In [72]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [73]:
inputs.invoke({"question": "bees?"})

{'context': [Document(metadata={}, page_content='bees are important for the environment'),
  Document(metadata={}, page_content='birds can fly'),
  Document(metadata={}, page_content='your mom is a doctor')],
 'question': 'bees?'}

## Bind

and OpenAI Functions

In [ ]:
functions = [ 
    { 
      "name": "get_factorial", 
      "description": "Calculate the factorial of a number", 
      "parameters": { 
        "type": "object", 
        "properties": { 
          "number": { 
            "type": "integer", 
            "description": "The number to calculate factorial of" 
          }, 
        }, 
        "required": ["number"] 
      } 
    } 
  ]

In [97]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt1 = ChatPromptTemplate.from_template(template)

In [98]:
prompt2 = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [99]:
runnable = prompt1 | prompt2 | model


In [101]:
runnable.invoke({"context": "", "question": "what is the factorial of 8?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"number":8}', 'name': 'get_factorial'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 162, 'total_tokens': 177, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bgqu115gBpw5uxMCWBHrwtxvVg21K', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--13b7c62c-2fcd-4a07-a387-e8f7c79d93cb-0', usage_metadata={'input_tokens': 162, 'output_tokens': 15, 'total_tokens': 177, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [102]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
    {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    },
    { 
      "name": "get_factorial", 
      "description": "Calculate the factorial of a number", 
      "parameters": { 
        "type": "object", 
        "properties": { 
          "number": { 
            "type": "integer", 
            "description": "The number to calculate factorial of" 
          }, 
        }, 
        "required": ["number"] 
      } 
    } 
  ]

In [103]:
model = model.bind(functions=functions)

In [104]:
runnable = prompt1 | prompt2 | model

In [108]:
runnable.invoke({"context" : "", "question": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 162, 'total_tokens': 180, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BgqvdXfgwYXJlB6LsXNHhKkl5IrU7', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--944a629d-5e2e-487a-9587-282b4bba95de-0', usage_metadata={'input_tokens': 162, 'output_tokens': 18, 'total_tokens': 180, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [109]:
runnable.invoke({"context" : "", "question":  "what is the weather in CA?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"CA"}', 'name': 'weather_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 161, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BgqvyQRrQM0uvRpZfq0XzcmRTPNkj', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--90732c73-2a86-43be-8440-bc5b9c485667-0', usage_metadata={'input_tokens': 161, 'output_tokens': 15, 'total_tokens': 176, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [110]:
runnable.invoke({"context" : "", "question": "what is the factorial of 9?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"number":9}', 'name': 'get_factorial'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 162, 'total_tokens': 177, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BgqwCCD5LF3zGexU1kR6ZjZFTitVp', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--d2b2290f-40ef-4796-aede-0812f6937445-0', usage_metadata={'input_tokens': 162, 'output_tokens': 15, 'total_tokens': 177, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [111]:
runnable.invoke({"context" : "", "question": "what is the name of indian president?"})

AIMessage(content="I'm sorry, but I cannot answer the question based on the provided context as it does not contain any information related to the name of the Indian President. If you have any specific information or context related to the Indian President, please provide it so that I can assist you with the answer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 162, 'total_tokens': 221, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BgqwNSSyed2LTa4NsfZ4htxxIZD4B', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--765e9c14-74fb-4bdc-b446-7079e0ae958d-0', usage_metadata={'input_tokens': 162, 'output_tokens': 59, 'total_tokens': 221, 'input_token_details':

## Fallbacks

In [112]:
from langchain_openai import OpenAI
import json

**Note**: Due to the deprication of OpenAI's model `text-davinci-001` on 4 January 2024, you'll be using OpenAI's recommended replacement model `gpt-3.5-turbo-instruct` instead.

In [113]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

In [114]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [115]:
simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [116]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 9 column 1 (char 125)

In [117]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [118]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

In [119]:
final_chain = simple_chain.with_fallbacks([chain])

In [120]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

## Interface

In [121]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a business idea about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [122]:
chain.invoke({"topic": "bears"})

'One possible business idea related to bears could be a "Bear Watching Tours" company. This business would offer guided tours to locations where customers can observe bears in their natural habitat. This could be in a national park, wildlife reserve, or even a bear sanctuary.\n\nThe tours could be tailored to different levels of interest and comfort, from beginner tours where customers observe bears from a safe distance, to more adventurous tours that involve hiking or camping to get up close and personal with these magnificent creatures.\n\nAdditionally, the company could offer educational components to teach customers about bear behavior, conservation efforts, and the importance of protecting bear habitats. They could also partner with local wildlife experts or researchers to provide in-depth knowledge and insights.\n\nOverall, this business idea would cater to nature enthusiasts, wildlife lovers, and tourists looking for unique and memorable experiences, while also promoting awarene

In [123]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

['One unique business idea related to bears could be starting a bear watching or photography tour company. This business could cater to nature enthusiasts and photography lovers who are interested in observing and capturing these majestic creatures in their natural habitat. Customers could join guided tours led by experienced naturalists and photographers to get up close and personal with bears in the wild, while also learning about their behaviors and conservation efforts. The tours could be offered in locations known for high bear populations, such as national parks or wildlife reserves. This business idea could provide not only a thrilling and educational experience for customers, but also contribute to wildlife conservation efforts by raising awareness and funds for bear protection.',
 'One business idea related to frogs could be a boutique store that specializes in unique and high-quality frog-themed products. This could include items such as handmade frog jewelry, home decor, clo

In [124]:
for t in chain.stream({"topic": "bears"}):
    print(t)


One
 possible
 business
 idea
 related
 to
 bears
 could
 be
 a
 bear
-themed
 cafe
 or
 restaurant
.
 This
 unique
 dining
 establishment
 could
 feature
 bear
-themed
 decor
,
 menu
 items
 with
 names
 inspired
 by
 bears
 (
such
 as
 the
 "
Gr
izzly
 Burger
"
 or
 "
P
olar
 Pl
unge
 Smooth
ie
"),
 and
 possibly
 even
 live
 bear
-themed
 entertainment
 (
such
 as
 bear
 mascot
 appearances
 or
 bear
 educational
 presentations
).
 This
 concept
 could
 appeal
 to
 animal
 lovers
,
 families
,
 and
 tourists
 looking
 for
 a
 fun
 and
 memorable
 dining
 experience
.
 Additionally
,
 a
 portion
 of
 the
 profits
 could
 be
 donated
 to
 bear
 conservation
 efforts
 or
 wildlife
 rehabilitation
 centers
.



In [125]:
response = await chain.ainvoke({"topic": "bears"})
response

'One business idea related to bears could be a bear sanctuary or wildlife rehabilitation center. This type of business would provide a safe and natural environment for rescued bears, either from the wild or from captivity, to live and thrive. The sanctuary could offer educational tours and programs for the public to learn about bears and their importance in the ecosystem, as well as opportunities for visitors to support conservation efforts through donations or volunteer work. Additionally, the sanctuary could partner with local schools and organizations to promote bear conservation and habitat preservation.'

In [126]:
response = await chain.abatch([{"topic": "bears"}, {"topic": "frogs"}])
response

['One business idea could be a bear tracking and education company. This company would offer guided tours in bear habitats, teaching clients about bears, their behavior, and how to safely observe them in the wild. The tours could also incorporate lessons on bear conservation and protection efforts. Additionally, the company could partner with wildlife researchers to collect data on bear populations and behaviors, providing valuable insights for conservation efforts. This business could cater to nature enthusiasts, wildlife photographers, and anyone interested in learning more about these majestic animals.',
 'One potential business idea involving frogs is creating a frog breeding and selling company. This business could specialize in breeding various species of frogs for pet owners, collectors, and educational institutions. The company could offer a wide range of frog species, including popular choices like tree frogs, dart frogs, and bullfrogs.\n\nAdditionally, the business could also

In [ ]:
for t in chain.astream({"topic": "bears"}):
    print(t)

TypeError: object async_generator can't be used in 'await' expression